In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import seaborn as sns
from keras.applications.vgg16 import VGG16
from keras.layers import Input
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.models import Model
from keras import backend as K
from keras.models import load_model
from sklearn.preprocessing import OneHotEncoder

In [9]:
train_rounds = [1,2,3]
test_round = [4]

X_train = []
y_train = []

for i in train_rounds:
    X_train.append(np.load('hand_activities_data/rounds/round{}_features_X.npy'.format(i)))
    y_train.append(np.load('hand_activities_data/rounds/round{}_features_labels.npy'.format(i)))

X_train = np.concatenate(X_train)
y_train = np.concatenate(y_train)

X_test = np.load('hand_activities_data/rounds/round{}_features_X.npy'.format(4))
y_test = np.load('hand_activities_data/rounds/round{}_features_labels.npy'.format(4))

In [10]:
ohe = OneHotEncoder()
y_train = ohe.fit_transform(y_train.reshape((-1,1))).toarray()
y_test = ohe.transform(y_test.reshape((-1,1))).toarray()

In [11]:
fft_size = 256
width = 48
num_channels = 3
num_classes = 25

X_train = X_train.reshape((-1, fft_size, width, num_channels))
X_test = X_test.reshape((-1, fft_size, width, num_channels))

In [14]:
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(93436, 64, 48, 3) (93436, 25)
(31203, 64, 48, 3) (31203, 25)


In [21]:
scale_factor = 8
X_train = X_train[:,0:fft_size//scale_factor,:,:]
X_test = X_test[:,0:fft_size//scale_factor,:,:]

In [23]:
inp = Input(shape = (fft_size//scale_factor, width, num_channels))

# Block-1
x = Conv2D(filters=64, kernel_size=(3,3), padding="same")(inp)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = MaxPooling2D(pool_size=(2,2),strides=(2,2))(x)

# Block-2
x = Conv2D(filters=128, kernel_size=(3,3), padding="valid")(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = MaxPooling2D(pool_size=(2,2),strides=(2,2))(x)

# Block-3
x = Conv2D(filters=256, kernel_size=(3,3), padding="valid")(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = MaxPooling2D(pool_size=(2,2),strides=(2,2))(x)

# # Block-4
# x = Conv2D(filters=512, kernel_size=(3,3), padding="valid")(x)
# x = BatchNormalization()(x)
# x = Activation('relu')(x)
# x = MaxPooling2D(pool_size=(2,2),strides=(2,2))(x)

x = Flatten()(x)

# Classifier
x = Dense(units=2000)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)

x = Dense(units=500)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dropout(0.4)(x)

out = Dense(units=num_classes, activation='softmax')(x)
model = Model(inp, out, name='hand_activities')

In [24]:
# model = load_model('model_main.h5')
model.summary()

Model: "hand_activities"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 32, 48, 3)]       0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 32, 48, 64)        1792      
_________________________________________________________________
batch_normalization_9 (Batch (None, 32, 48, 64)        256       
_________________________________________________________________
activation_9 (Activation)    (None, 32, 48, 64)        0         
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 16, 24, 64)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 14, 22, 128)       73856     
_________________________________________________________________
batch_normalization_10 (Batc (None, 14, 22, 128)   

In [25]:
model_name = 'model_{}'.format(fft_size//2//scale_factor)
print(model_name)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics='accuracy')
model_checkpoint = ModelCheckpoint(filepath='checkpoints/{}.h5'.format(model_name), 
                                   monitor='val_loss', verbose=1, save_best_only=True)
early_stopping = EarlyStopping(monitor='val_accuracy', verbose=1, patience=12)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=7, verbose=1, min_lr=10e-5)

model_16


In [26]:
history = model.fit(X_train, y_train,
                    validation_data=(X_test, y_test),
                    batch_size=64,
                    epochs=100,
                    shuffle=True, 
                    callbacks=[model_checkpoint, early_stopping, reduce_lr],
                    )

Epoch 1/100
1460/1460 [==============================] - ETA: 0s - loss: 0.8601 - accuracy: 0.6983
Epoch 00001: val_loss improved from inf to 1.21039, saving model to checkpoints/model_16.h5
1460/1460 [==============================] - 13s 9ms/step - loss: 0.8601 - accuracy: 0.6983 - val_loss: 1.2104 - val_accuracy: 0.6314
Epoch 2/100
1457/1460 [============================>.] - ETA: 0s - loss: 0.3971 - accuracy: 0.8544
Epoch 00002: val_loss did not improve from 1.21039
1460/1460 [==============================] - 12s 8ms/step - loss: 0.3970 - accuracy: 0.8544 - val_loss: 1.4353 - val_accuracy: 0.6712
Epoch 3/100
1455/1460 [============================>.] - ETA: 0s - loss: 0.2580 - accuracy: 0.9053
Epoch 00003: val_loss did not improve from 1.21039
1460/1460 [==============================] - 12s 8ms/step - loss: 0.2578 - accuracy: 0.9054 - val_loss: 1.2173 - val_accuracy: 0.6856
Epoch 4/100
1457/1460 [============================>.] - ETA: 0s - loss: 0.1862 - accuracy: 0.9313
Epoch 00

In [27]:
model = load_model('checkpoints/{}.h5'.format(model_name))

In [28]:
print(model_name)
model.evaluate(X_test, y_test)

model_16
976/976 [==============================] - 2s 2ms/step - loss: 0.9157 - accuracy: 0.8209


[0.9157345294952393, 0.8208826184272766]